In [2]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import json
from dpm.modules.search_strategy import beam_search

/root/.conda/envs/pic2text/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def Pic2Text(Model, image_path):
    # 读取图像
    image = Image.open(image_path)
    
    # 转换图像
    transform = transforms.Compose([
        transforms.Resize((512, 374)),
        transforms.ToTensor(),
    ])

    transformed_image = transform(image)

    reshaped_image = transformed_image.unsqueeze(0)

    # 生成序列
    result,_ = beam_search(model=Model, X=reshaped_image, beam_width=3,predictions=60)

    # 获取index
    Index_file_path = "data/vocabulary.json"

    with open(Index_file_path, 'r') as json_file:
        index = json.load(json_file)
    
    index = dict(zip(index.values(), index.keys()))

    # 展示句子
    gen_num = result.size(1)

    print(f"{image_path} 生成句子为：")
    for i in range(gen_num):
        description = result[0, gen_num].tolist()
        description = ' '.join([index[w] for w in description if w not in range(4)])
        print(description)

In [4]:
model=torch.load('/root/pic2text/logs/2023-12-28T09-12-33/checkpoints/save.pt')

In [5]:
img_path='/root/pic2text/data/deepfashion-multimodal/images/MEN-Denim-id_00000938-02_1_front.jpg'

In [6]:
Pic2Text(model,img_path)